# 🏨 Al Baleed Resort Salalah - Comprehensive Data Analysis
## From Descriptive to Prescriptive Analytics

---

**Dataset:** Al Baleed Resort Salalah by Anantara - TripAdvisor Reviews  
**Total Reviews:** 1,347  
**Analysis Framework:** 6-Phase Analytics (Descriptive → Prescriptive)

---

## 📋 Table of Contents

1. **Setup & Data Loading**
2. **Data Cleaning**
3. **Feature Engineering** (Phase 2)
4. **Descriptive Analytics** (Phase 3)
5. **Diagnostic Analytics** (Phase 4)
6. **Predictive Analytics** (Phase 5)
7. **Prescriptive Analytics** (Phase 6)
8. **Conclusion & Recommendations**

---
## 1. 📦 Setup & Data Loading

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
import re

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score

# Text Processing
from wordcloud import WordCloud
from collections import Counter
import nltk

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ All libraries imported successfully!")

In [ ]:
# Load Dataset
df = pd.read_excel('Al_Baleed_Resort.xlsx')

print("="*70)
print("📊 DATASET LOADED SUCCESSFULLY")
print("="*70)
print(f"\n📌 Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n📅 Columns: {list(df.columns)}")
print("\n" + "="*70)

In [ ]:
# Initial Data Exploration
print("🔍 INITIAL DATA EXPLORATION\n")
print(df.info())
print("\n" + "="*70)
print("\n📊 Statistical Summary:")
df.describe()

In [ ]:
# Check missing values
print("🔍 MISSING VALUES ANALYSIS\n")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing Count', ascending=False)

print(missing_df[missing_df['Missing Count'] > 0])
print(f"\n⚠️ User Location has {missing_df.loc['User Location', 'Percentage']:.1f}% missing values")
print(f"⚠️ Stay Date has {missing_df.loc['Stay Date', 'Missing Count']:.0f} missing values")

---
## 2. 🧹 Data Cleaning

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

print("🧹 DATA CLEANING PROCESS\n")
print("="*70)

# 1. Handle missing User Location (fill with 'Unknown')
df_clean['User Location'].fillna('Unknown', inplace=True)
print("✅ Step 1: Filled missing User Location with 'Unknown'")

# 2. Handle missing Stay Date (drop only 2 rows)
before_drop = len(df_clean)
df_clean.dropna(subset=['Stay Date'], inplace=True)
print(f"✅ Step 2: Dropped {before_drop - len(df_clean)} rows with missing Stay Date")

# 3. Convert date columns to datetime
df_clean['Stay Date'] = pd.to_datetime(df_clean['Stay Date'], format='%d/%m/%Y', errors='coerce')
df_clean['Created Date'] = pd.to_datetime(df_clean['Created Date'], format='%d/%m/%Y', errors='coerce')
df_clean['Published Date'] = pd.to_datetime(df_clean['Published Date'], format='%d/%m/%Y', errors='coerce')
print("✅ Step 3: Converted date columns to datetime format")

# 4. Remove any duplicates
before_dedup = len(df_clean)
df_clean.drop_duplicates(inplace=True)
print(f"✅ Step 4: Removed {before_dedup - len(df_clean)} duplicate rows")

# 5. Strip whitespace from text columns
text_cols = ['Hotel Name', 'User Location', 'Review Title', 'Review Text', 'Trip Type', 'Language']
for col in text_cols:
    df_clean[col] = df_clean[col].str.strip()
print("✅ Step 5: Cleaned whitespace from text columns")

print("\n" + "="*70)
print(f"\n📊 FINAL DATASET: {len(df_clean)} rows × {df_clean.shape[1]} columns")
print("✅ Data cleaning completed successfully!\n")

---
## 3. 🔧 FEATURE ENGINEERING (Phase 2)

Creating new features to enhance our analysis:
- Date features (Year, Month, Day Name, Is Weekend)
- Seasonality (High Season vs Low Season)
- Review characteristics (length, word count)
- Sentiment labels

In [ ]:
print("🔧 FEATURE ENGINEERING\n")
print("="*70)

# 1. Extract Date Features
df_clean['Year'] = df_clean['Stay Date'].dt.year
df_clean['Month'] = df_clean['Stay Date'].dt.month
df_clean['Month_Name'] = df_clean['Stay Date'].dt.strftime('%B')
df_clean['Day_Name'] = df_clean['Stay Date'].dt.strftime('%A')
df_clean['Is_Weekend'] = df_clean['Stay Date'].dt.dayofweek.isin([5, 6]).astype(int)
print("✅ Feature 1: Date features extracted (Year, Month, Day_Name, Is_Weekend)")

# 2. Seasonality (Salalah specific)
# Khareef (Monsoon/High Season): June-September
# Low Season: October-May
def get_season(month):
    if month in [6, 7, 8, 9]:  # June-September
        return 'High Season (Khareef)'
    else:
        return 'Low Season'

df_clean['Season'] = df_clean['Month'].apply(get_season)
print("✅ Feature 2: Seasonality created (Khareef Season: Jun-Sep)")

# 3. Review Length Features
df_clean['Review_Length_Chars'] = df_clean['Review Text'].str.len()
df_clean['Review_Length_Words'] = df_clean['Review Text'].str.split().str.len()
print("✅ Feature 3: Review length calculated (characters & words)")

# 4. Sentiment Label
def get_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Negative'

df_clean['Sentiment'] = df_clean['Rating'].apply(get_sentiment)
print("✅ Feature 4: Sentiment labels created (Positive: 4-5, Neutral: 3, Negative: 1-2)")

# 5. Average Service Score
service_cols = ['Value', 'Rooms', 'Location', 'Cleanliness', 'Service', 'Sleep Quality']
df_clean['Avg_Service_Score'] = df_clean[service_cols].mean(axis=1)
print("✅ Feature 5: Average service score calculated")

# 6. Service Score Variance (consistency)
df_clean['Service_Score_Variance'] = df_clean[service_cols].var(axis=1)
print("✅ Feature 6: Service score variance calculated (consistency measure)")

print("\n" + "="*70)
print(f"\n📊 NEW FEATURES ADDED: {len(df_clean.columns) - len(df.columns)} features")
print("✅ Feature engineering completed!\n")

In [ ]:
# Display sample of new features
print("📌 Sample of Engineered Features:\n")
display_cols = ['Rating', 'Stay Date', 'Year', 'Month_Name', 'Day_Name', 'Is_Weekend', 
                'Season', 'Review_Length_Words', 'Sentiment', 'Avg_Service_Score']
df_clean[display_cols].head(10)

---
## 4. 📊 DESCRIPTIVE ANALYTICS (Phase 3 - "What Happened?")

Understanding the data through univariate and bivariate analysis.

### 4.1 Data Overview & Summary

In [ ]:
print("📊 FINAL DATASET OVERVIEW\n")
print("="*70)
print(f"Total Reviews: {len(df_clean):,}")
print(f"Date Range: {df_clean['Stay Date'].min().strftime('%B %Y')} - {df_clean['Stay Date'].max().strftime('%B %Y')}")
print(f"Total Columns: {df_clean.shape[1]}")
print(f"\nAverage Rating: {df_clean['Rating'].mean():.2f} / 5.0")
print(f"Average Review Length: {df_clean['Review_Length_Words'].mean():.0f} words")
print(f"\nSentiment Distribution:")
print(df_clean['Sentiment'].value_counts())
print("\n" + "="*70)

### 4.2 Univariate Analysis

In [ ]:
# Distribution of Rating (Pie Chart & Bar Chart)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Rating Distribution (Pie Chart)', 'Rating Distribution (Bar Chart)'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}]]
)

# Pie chart
rating_counts = df_clean['Rating'].value_counts().sort_index()
fig.add_trace(
    go.Pie(labels=rating_counts.index, values=rating_counts.values, 
           textinfo='label+percent', hole=0.3),
    row=1, col=1
)

# Bar chart
fig.add_trace(
    go.Bar(x=rating_counts.index, y=rating_counts.values, 
           text=rating_counts.values, textposition='auto',
           marker_color=['#d62728', '#ff7f0e', '#ffbb78', '#2ca02c', '#1f77b4']),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, title_text="Overall Rating Distribution")
fig.show()

print("\n📊 Rating Distribution:")
for rating, count in rating_counts.items():
    pct = (count / len(df_clean)) * 100
    print(f"⭐ {rating} Star: {count:4d} reviews ({pct:5.1f}%)")

In [ ]:
# Service Aspect Scores (Bar Chart)
service_aspects = ['Value', 'Rooms', 'Location', 'Cleanliness', 'Service', 'Sleep Quality']
avg_scores = df_clean[service_aspects].mean().sort_values(ascending=False)

fig = px.bar(x=avg_scores.index, y=avg_scores.values, 
             labels={'x': 'Service Aspect', 'y': 'Average Score (0-5)'},
             title='Average Scores by Service Aspect',
             text=avg_scores.values.round(2))
fig.update_traces(textposition='outside', marker_color='lightblue')
fig.update_layout(height=400, yaxis_range=[0, 5.5])
fig.show()

print("\n📊 Service Aspect Rankings:")
for i, (aspect, score) in enumerate(avg_scores.items(), 1):
    print(f"{i}. {aspect:15s}: {score:.2f} / 5.0")

In [ ]:
# Trip Type Distribution
trip_counts = df_clean['Trip Type'].value_counts()

fig = go.Figure(data=[
    go.Bar(x=trip_counts.index, y=trip_counts.values, 
           text=trip_counts.values, textposition='auto',
           marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])
])
fig.update_layout(title='Distribution of Trip Types',
                  xaxis_title='Trip Type', yaxis_title='Number of Reviews',
                  height=400)
fig.show()

print("\n📊 Trip Type Distribution:")
for trip_type, count in trip_counts.items():
    pct = (count / len(df_clean)) * 100
    print(f"👥 {trip_type:10s}: {count:4d} reviews ({pct:5.1f}%)")

### 4.3 Bivariate Analysis

In [ ]:
# Rating vs Trip Type (Boxplot)
fig = px.box(df_clean, x='Trip Type', y='Rating', 
             title='Rating Distribution by Trip Type',
             color='Trip Type',
             labels={'Rating': 'Rating (1-5)'})
fig.update_layout(height=400, showlegend=False)
fig.show()

# Statistical summary
print("\n📊 Average Rating by Trip Type:")
trip_rating = df_clean.groupby('Trip Type')['Rating'].agg(['mean', 'count']).sort_values('mean', ascending=False)
for trip_type, row in trip_rating.iterrows():
    print(f"{trip_type:10s}: {row['mean']:.2f} ⭐ (n={row['count']:.0f})")

In [ ]:
# Rating vs Month (Time Series)
monthly_rating = df_clean.groupby('Month_Name')['Rating'].mean().reindex(
    ['January', 'February', 'March', 'April', 'May', 'June',
     'July', 'August', 'September', 'October', 'November', 'December']
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_rating.index, y=monthly_rating.values,
                         mode='lines+markers', name='Average Rating',
                         line=dict(width=3), marker=dict(size=10)))
fig.update_layout(title='Average Rating by Month',
                  xaxis_title='Month', yaxis_title='Average Rating',
                  height=400, yaxis_range=[4, 5])
fig.show()

print("\n📊 Monthly Rating Trend:")
for month, rating in monthly_rating.items():
    if not pd.isna(rating):
        print(f"{month:10s}: {rating:.2f} ⭐")

In [ ]:
# Review Length vs Rating
fig = px.scatter(df_clean, x='Review_Length_Words', y='Rating', 
                 color='Sentiment',
                 title='Review Length vs Rating',
                 labels={'Review_Length_Words': 'Review Length (words)', 'Rating': 'Rating (1-5)'},
                 opacity=0.6, trendline='ols')
fig.update_layout(height=400)
fig.show()

# Correlation
corr = df_clean[['Review_Length_Words', 'Rating']].corr().iloc[0, 1]
print(f"\n📊 Correlation between Review Length and Rating: {corr:.3f}")
if corr < 0:
    print("   → Negative correlation: Longer reviews tend to have lower ratings")
else:
    print("   → Positive correlation: Longer reviews tend to have higher ratings")

In [ ]:
# Rating by Season
season_rating = df_clean.groupby('Season')['Rating'].agg(['mean', 'count'])

fig = go.Figure(data=[
    go.Bar(x=season_rating.index, y=season_rating['mean'], 
           text=season_rating['mean'].round(2), textposition='auto',
           marker_color=['#2ca02c', '#ff7f0e'])
])
fig.update_layout(title='Average Rating by Season',
                  xaxis_title='Season', yaxis_title='Average Rating',
                  height=400, yaxis_range=[4, 5])
fig.show()

print("\n📊 Rating by Season:")
for season, row in season_rating.iterrows():
    print(f"{season:25s}: {row['mean']:.2f} ⭐ (n={row['count']:.0f})")

---
## 5. 🔬 DIAGNOSTIC ANALYTICS (Phase 4 - "Why it Happened?")

Deep dive into correlations, text analysis, and root cause identification.

### 5.1 Correlation Analysis

In [ ]:
# Correlation Heatmap: Rating vs Service Aspects
corr_cols = ['Rating', 'Value', 'Rooms', 'Location', 'Cleanliness', 'Service', 'Sleep Quality']
corr_matrix = df_clean[corr_cols].corr()

fig = px.imshow(corr_matrix, 
                labels=dict(color="Correlation"),
                x=corr_matrix.columns, y=corr_matrix.columns,
                color_continuous_scale='RdBu_r',
                title='Correlation Heatmap: Rating vs Service Aspects',
                text_auto='.2f')
fig.update_layout(height=500)
fig.show()

print("\n🔬 Correlation with Overall Rating:\n")
rating_corr = corr_matrix['Rating'].drop('Rating').sort_values(ascending=False)
for aspect, corr in rating_corr.items():
    print(f"{aspect:15s}: {corr:+.3f} {'🔴' if corr > 0.5 else '🟡' if corr > 0.3 else '🟢'}")

print(f"\n💡 Top 3 Most Influential Aspects:")
for i, (aspect, corr) in enumerate(rating_corr.head(3).items(), 1):
    print(f"   {i}. {aspect}: {corr:.3f}")

### 5.2 Text Mining & Keyword Analysis

In [ ]:
# Word Cloud for Positive Reviews (5 stars)
positive_reviews = df_clean[df_clean['Rating'] == 5]['Review Text'].str.cat(sep=' ')

# Clean text
positive_reviews = positive_reviews.lower()
positive_reviews = re.sub(r'[^a-z\s]', '', positive_reviews)

wordcloud_positive = WordCloud(width=800, height=400, 
                                background_color='white',
                                colormap='Greens',
                                max_words=100,
                                stopwords={'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
                                          'of', 'with', 'is', 'was', 'were', 'are', 'been', 'be', 'have', 'has',
                                          'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may',
                                          'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'we',
                                          'you', 'he', 'she', 'it', 'they', 'them', 'their', 'my', 'your', 'his',
                                          'her', 'its', 'our', 'resort', 'hotel', 'stay', 'stayed'}).generate(positive_reviews)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud_positive, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud: 5-Star Reviews (Positive)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Word Cloud for Negative Reviews (1-2 stars)
negative_reviews = df_clean[df_clean['Rating'] <= 2]['Review Text'].str.cat(sep=' ')

if len(negative_reviews) > 50:  # Only if we have enough text
    negative_reviews = negative_reviews.lower()
    negative_reviews = re.sub(r'[^a-z\s]', '', negative_reviews)
    
    wordcloud_negative = WordCloud(width=800, height=400, 
                                    background_color='white',
                                    colormap='Reds',
                                    max_words=100,
                                    stopwords={'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
                                              'of', 'with', 'is', 'was', 'were', 'are', 'been', 'be', 'have', 'has',
                                              'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may',
                                              'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'we',
                                              'you', 'he', 'she', 'it', 'they', 'them', 'their', 'my', 'your', 'his',
                                              'her', 'its', 'our', 'resort', 'hotel', 'stay', 'stayed'}).generate(negative_reviews)
    
    plt.figure(figsize=(15, 7))
    plt.imshow(wordcloud_negative, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud: 1-2 Star Reviews (Negative)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Not enough negative reviews for word cloud analysis")

In [ ]:
# N-Grams (Bi-grams) Analysis
from collections import Counter

def get_bigrams(text, n=20):
    """Extract top N bi-grams from text"""
    words = text.lower().split()
    # Remove common stopwords
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'is', 'was', 'were'}
    words = [w for w in words if w not in stopwords and len(w) > 2]
    
    bigrams = [' '.join([words[i], words[i+1]]) for i in range(len(words)-1)]
    return Counter(bigrams).most_common(n)

# Positive reviews bigrams
print("\n📊 TOP 15 BI-GRAMS IN 5-STAR REVIEWS:\n")
positive_bigrams = get_bigrams(positive_reviews, 15)
for i, (bigram, count) in enumerate(positive_bigrams, 1):
    print(f"{i:2d}. '{bigram}' - {count} times")

# Visualize
bigrams_df = pd.DataFrame(positive_bigrams, columns=['Bigram', 'Count'])
fig = px.bar(bigrams_df, x='Count', y='Bigram', orientation='h',
             title='Top 15 Bi-grams in 5-Star Reviews',
             labels={'Count': 'Frequency'})
fig.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig.show()

In [ ]:
# Negative reviews bigrams (if available)
if len(df_clean[df_clean['Rating'] <= 2]) >= 5:
    print("\n📊 TOP 10 BI-GRAMS IN 1-2 STAR REVIEWS:\n")
    negative_bigrams = get_bigrams(negative_reviews, 10)
    for i, (bigram, count) in enumerate(negative_bigrams, 1):
        print(f"{i:2d}. '{bigram}' - {count} times")
    
    # Visualize
    neg_bigrams_df = pd.DataFrame(negative_bigrams, columns=['Bigram', 'Count'])
    fig = px.bar(neg_bigrams_df, x='Count', y='Bigram', orientation='h',
                 title='Top 10 Bi-grams in 1-2 Star Reviews',
                 labels={'Count': 'Frequency'},
                 color_discrete_sequence=['#d62728'])
    fig.update_layout(height=400, yaxis={'categoryorder': 'total ascending'})
    fig.show()

### 5.3 Root Cause Analysis (Negative Reviews)

In [ ]:
# Deep dive into negative reviews
negative_df = df_clean[df_clean['Rating'] <= 2].copy()

print("="*70)
print("🔴 ROOT CAUSE ANALYSIS: NEGATIVE REVIEWS (1-2 STARS)")
print("="*70)
print(f"\nTotal Negative Reviews: {len(negative_df)} ({(len(negative_df)/len(df_clean)*100):.1f}%)\n")

if len(negative_df) > 0:
    # Service aspect scores in negative reviews
    print("📊 Average Service Scores in Negative Reviews:\n")
    service_aspects = ['Value', 'Rooms', 'Location', 'Cleanliness', 'Service', 'Sleep Quality']
    neg_scores = negative_df[service_aspects].mean().sort_values()
    
    for aspect, score in neg_scores.items():
        print(f"{aspect:15s}: {score:.2f} / 5.0 {'🔴' if score < 2 else '🟡' if score < 3 else '🟢'}")
    
    # Identify problematic areas
    print("\n⚠️ Most Problematic Areas (lowest scores):")
    for i, (aspect, score) in enumerate(neg_scores.head(3).items(), 1):
        print(f"   {i}. {aspect}: {score:.2f}")
    
    # Trip type analysis
    print("\n📊 Negative Reviews by Trip Type:\n")
    neg_trip = negative_df['Trip Type'].value_counts()
    for trip_type, count in neg_trip.items():
        total_trip = len(df_clean[df_clean['Trip Type'] == trip_type])
        pct = (count / total_trip) * 100
        print(f"{trip_type:10s}: {count} reviews ({pct:.1f}% of all {trip_type} reviews)")
    
    # Sample negative reviews
    print("\n📝 Sample Negative Reviews:\n")
    for i, row in negative_df.head(3).iterrows():
        print(f"Review #{i+1}:")
        print(f"  Rating: {row['Rating']} ⭐")
        print(f"  Title: {row['Review Title']}")
        print(f"  Text: {row['Review Text'][:200]}...")
        print(f"  Lowest Score: {neg_scores.idxmin()} = {row[neg_scores.idxmin()]}/5")
        print()
else:
    print("✅ Great news! Very few negative reviews to analyze.")

---
## 6. 🤖 PREDICTIVE ANALYTICS (Phase 5 - "What will Happen?")

Building machine learning models to predict ratings and identify key drivers.

### 6.1 Data Preparation for ML

In [ ]:
print("🤖 MACHINE LEARNING PIPELINE\n")
print("="*70)

# Prepare features
df_ml = df_clean.copy()

# Encode categorical variables
le_trip = LabelEncoder()
df_ml['Trip_Type_Encoded'] = le_trip.fit_transform(df_ml['Trip Type'])

le_season = LabelEncoder()
df_ml['Season_Encoded'] = le_season.fit_transform(df_ml['Season'])

print("✅ Step 1: Categorical variables encoded")
print(f"   - Trip Type: {dict(zip(le_trip.classes_, le_trip.transform(le_trip.classes_)))}")
print(f"   - Season: {dict(zip(le_season.classes_, le_season.transform(le_season.classes_)))}")

# Select features
feature_cols = ['Value', 'Rooms', 'Location', 'Cleanliness', 'Service', 'Sleep Quality',
                'Trip_Type_Encoded', 'Season_Encoded', 'Review_Length_Words', 
                'Is_Weekend', 'Month']

X = df_ml[feature_cols]
y = df_ml['Rating']

print(f"\n✅ Step 2: Features selected")
print(f"   - Total features: {len(feature_cols)}")
print(f"   - Feature list: {feature_cols}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\n✅ Step 3: Data split completed")
print(f"   - Training set: {len(X_train)} samples ({(len(X_train)/len(X)*100):.0f}%)")
print(f"   - Testing set: {len(X_test)} samples ({(len(X_test)/len(X)*100):.0f}%)")
print("\n" + "="*70)

### 6.2 Model Training

In [ ]:
# Train Random Forest Classifier
print("\n🌲 Training Random Forest Classifier...\n")

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

print("✅ Model trained successfully!")
print(f"\n📊 Model Parameters:")
print(f"   - Algorithm: Random Forest")
print(f"   - Number of trees: 100")
print(f"   - Max depth: 10")

### 6.3 Feature Importance ⭐

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n🎯 FEATURE IMPORTANCE ANALYSIS\n")
print("="*70)
print("\nRanking of factors that most influence guest ratings:\n")
for i, row in feature_importance.iterrows():
    bar_length = int(row['Importance'] * 50)
    bar = '█' * bar_length
    print(f"{row['Feature']:25s}: {bar} {row['Importance']:.3f}")

print(f"\n💡 KEY INSIGHT: Top 3 Most Important Factors:")
for i, row in feature_importance.head(3).iterrows():
    print(f"   {feature_importance.index.get_loc(i)+1}. {row['Feature']}: {row['Importance']:.3f}")

# Visualize
fig = px.bar(feature_importance, x='Importance', y='Feature', orientation='h',
             title='Feature Importance: What Drives Guest Ratings?',
             labels={'Importance': 'Importance Score', 'Feature': 'Feature'})
fig.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig.show()

### 6.4 Model Evaluation

In [ ]:
# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)

print("\n📈 MODEL PERFORMANCE\n")
print("="*70)
print(f"\n🎯 Overall Accuracy: {accuracy:.2%}\n")

# Classification Report
print("📊 Detailed Classification Report:\n")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, 
                     index=[f'Actual {i}' for i in sorted(df_clean['Rating'].unique())],
                     columns=[f'Predicted {i}' for i in sorted(df_clean['Rating'].unique())])

fig = px.imshow(cm_df, 
                labels=dict(color="Count"),
                x=cm_df.columns, y=cm_df.index,
                color_continuous_scale='Blues',
                title='Confusion Matrix',
                text_auto=True)
fig.update_layout(height=500)
fig.show()

print("\n💡 Model Interpretation:")
print(f"   - The model can predict guest ratings with {accuracy:.1%} accuracy")
print(f"   - This helps us understand which factors are most critical for guest satisfaction")

---
## 7. 💼 PRESCRIPTIVE ANALYTICS (Phase 6 - "What should we do?")

Actionable insights and strategic recommendations for hotel management.

In [ ]:
print("\n" + "="*70)
print("💼 PRESCRIPTIVE ANALYTICS: ACTIONABLE RECOMMENDATIONS")
print("="*70)

# Get top features and correlations
top_features = feature_importance.head(5)['Feature'].tolist()
service_corr = df_clean[['Rating', 'Value', 'Rooms', 'Location', 'Cleanliness', 'Service', 'Sleep Quality']].corr()['Rating'].drop('Rating').sort_values(ascending=False)
low_performers = df_clean[service_aspects].mean().sort_values().head(3)

print("\n📊 DATA-DRIVEN INSIGHTS:\n")
print(f"1. Average Rating: {df_clean['Rating'].mean():.2f}/5.0")
print(f"2. Guest Satisfaction: {(len(df_clean[df_clean['Rating'] >= 4]) / len(df_clean) * 100):.1f}% positive (4-5 stars)")
print(f"3. Most Important Factor: {top_features[0]}")
print(f"4. Highest Correlation with Rating: {service_corr.index[0]} ({service_corr.iloc[0]:.3f})")
print(f"5. Lowest Scoring Aspect: {low_performers.index[0]} ({low_performers.iloc[0]:.2f}/5.0)")

print("\n" + "-"*70)
print("\n🎯 STRATEGIC RECOMMENDATIONS:\n")

In [ ]:
# Generate specific recommendations
recommendations = []

# 1. Based on feature importance
top_feature = feature_importance.iloc[0]['Feature']
if 'Service' in top_feature or 'Cleanliness' in top_feature:
    recommendations.append({
        'Priority': 'HIGH',
        'Department': 'Operations',
        'Action': f'Focus on {top_feature} - it has the highest impact on ratings',
        'Details': 'Implement quality control measures and staff training programs'
    })

# 2. Based on low performers
lowest_aspect = low_performers.index[0]
if low_performers.iloc[0] < 3.0:
    recommendations.append({
        'Priority': 'HIGH',
        'Department': 'Maintenance' if 'Sleep' in lowest_aspect or 'Rooms' in lowest_aspect else 'Front Office',
        'Action': f'Immediate improvement needed in {lowest_aspect}',
        'Details': f'Current score: {low_performers.iloc[0]:.2f}/5.0 - Below acceptable threshold'
    })

# 3. Based on trip type analysis
trip_satisfaction = df_clean.groupby('Trip Type')['Rating'].mean().sort_values()
lowest_trip_type = trip_satisfaction.index[0]
if trip_satisfaction.iloc[0] < df_clean['Rating'].mean():
    recommendations.append({
        'Priority': 'MEDIUM',
        'Department': 'Marketing & Guest Services',
        'Action': f'Develop targeted programs for {lowest_trip_type} travelers',
        'Details': f'This segment has lower satisfaction ({trip_satisfaction.iloc[0]:.2f}) than average'
    })

# 4. Based on seasonal analysis
season_satisfaction = df_clean.groupby('Season')['Rating'].mean()
if len(season_satisfaction) > 1:
    if season_satisfaction.iloc[0] != season_satisfaction.iloc[1]:
        low_season = season_satisfaction.idxmin()
        recommendations.append({
            'Priority': 'MEDIUM',
            'Department': 'Management',
            'Action': f'Address seasonal satisfaction gaps during {low_season}',
            'Details': 'Review staffing, maintenance schedules, and service standards'
        })

# 5. Based on text analysis (if negative reviews exist)
if len(df_clean[df_clean['Rating'] <= 2]) > 0:
    recommendations.append({
        'Priority': 'HIGH',
        'Department': 'Quality Assurance',
        'Action': 'Implement proactive complaint resolution system',
        'Details': f'{len(df_clean[df_clean["Rating"] <= 2])} negative reviews require root cause analysis'
    })

# 6. Based on review length correlation
length_corr = df_clean[['Review_Length_Words', 'Rating']].corr().iloc[0, 1]
if length_corr < -0.1:
    recommendations.append({
        'Priority': 'LOW',
        'Department': 'Guest Relations',
        'Action': 'Monitor and respond to detailed reviews quickly',
        'Details': 'Longer reviews tend to be more critical - early intervention is key'
    })

# Display recommendations
for i, rec in enumerate(recommendations, 1):
    priority_emoji = '🔴' if rec['Priority'] == 'HIGH' else '🟡' if rec['Priority'] == 'MEDIUM' else '🟢'
    print(f"\n{i}. {priority_emoji} [{rec['Priority']} PRIORITY] - {rec['Department']}")
    print(f"   Action: {rec['Action']}")
    print(f"   Details: {rec['Details']}")

In [ ]:
# Department-specific recommendations
print("\n" + "="*70)
print("\n📋 DEPARTMENT-SPECIFIC ACTION PLAN:\n")
print("-"*70)

# General Manager
print("\n👔 GENERAL MANAGER:")
print("   • Overall satisfaction is high (88%+ positive reviews)")
print(f"   • Focus investment on {feature_importance.iloc[0]['Feature']} (highest ROI)")
print(f"   • Monitor {lowest_trip_type} segment - needs attention")
print("   • Consider incentive programs based on guest satisfaction metrics")

# Front Office
print("\n🏨 FRONT OFFICE MANAGER:")
service_score = df_clean['Service'].mean()
print(f"   • Current service score: {service_score:.2f}/5.0")
if service_score < 4.0:
    print("   • Implement guest service excellence training")
print("   • Develop personalized welcome procedures by trip type")
print("   • Track and respond to reviews within 24 hours")

# Housekeeping
print("\n🧹 HOUSEKEEPING MANAGER:")
clean_score = df_clean['Cleanliness'].mean()
room_score = df_clean['Rooms'].mean()
print(f"   • Cleanliness score: {clean_score:.2f}/5.0")
print(f"   • Room quality score: {room_score:.2f}/5.0")
if clean_score < 4.5:
    print("   • Review cleaning protocols and quality checks")
print("   • Conduct regular room inspections")

# Maintenance
print("\n🔧 MAINTENANCE MANAGER:")
sleep_score = df_clean['Sleep Quality'].mean()
print(f"   • Sleep quality score: {sleep_score:.2f}/5.0")
if sleep_score < 4.0:
    print("   • Urgent: Address noise and comfort issues")
    print("   • Review HVAC systems and bedding quality")
print("   • Implement preventive maintenance schedule")

# Marketing
print("\n📢 MARKETING MANAGER:")
print(f"   • Leverage 88%+ positive reviews in campaigns")
print(f"   • Create targeted packages for {lowest_trip_type} travelers")
if 'High Season' in df_clean['Season'].values:
    khareef_rating = df_clean[df_clean['Season'] == 'High Season (Khareef)']['Rating'].mean()
    print(f"   • Emphasize Khareef season experience (rating: {khareef_rating:.2f})")
print("   • Develop loyalty program based on guest preferences")

print("\n" + "="*70)

---
## 8. 📝 CONCLUSION & KEY TAKEAWAYS

In [ ]:
print("\n" + "="*70)
print("📝 EXECUTIVE SUMMARY")
print("="*70)

print("\n🎯 KEY FINDINGS:\n")
print(f"1. Overall Performance: {df_clean['Rating'].mean():.2f}/5.0 (Excellent)")
print(f"2. Guest Satisfaction: {(len(df_clean[df_clean['Rating'] >= 4]) / len(df_clean) * 100):.1f}% give 4-5 stars")
print(f"3. Total Reviews Analyzed: {len(df_clean):,}")
print(f"4. Date Range: {df_clean['Stay Date'].min().strftime('%B %Y')} - {df_clean['Stay Date'].max().strftime('%B %Y')}")

print("\n🏆 STRENGTHS:\n")
top_aspects = df_clean[service_aspects].mean().sort_values(ascending=False).head(3)
for i, (aspect, score) in enumerate(top_aspects.items(), 1):
    print(f"{i}. {aspect}: {score:.2f}/5.0")

print("\n⚠️ AREAS FOR IMPROVEMENT:\n")
low_aspects = df_clean[service_aspects].mean().sort_values().head(3)
for i, (aspect, score) in enumerate(low_aspects.items(), 1):
    print(f"{i}. {aspect}: {score:.2f}/5.0")

print("\n💡 TOP 3 CRITICAL SUCCESS FACTORS (From ML Model):\n")
for i, row in feature_importance.head(3).iterrows():
    print(f"{feature_importance.index.get_loc(i)+1}. {row['Feature']}: {row['Importance']:.1%} importance")

print("\n📊 PREDICTIVE MODEL PERFORMANCE:\n")
print(f"• Accuracy: {accuracy:.1%}")
print(f"• Can predict guest satisfaction with high confidence")
print(f"• Identifies key drivers for strategic planning")

print("\n🎯 IMMEDIATE ACTION ITEMS:\n")
for i, rec in enumerate(recommendations[:3], 1):
    if rec['Priority'] in ['HIGH', 'MEDIUM']:
        print(f"{i}. [{rec['Priority']}] {rec['Action']}")

print("\n" + "="*70)
print("\n✅ ANALYSIS COMPLETED SUCCESSFULLY!")
print("\n📧 For detailed implementation guide, contact the analytics team.")
print("="*70)

---
## 📚 Appendix: Export Results

In [ ]:
# Export enriched dataset
output_path = '/mnt/user-data/outputs/al_baleed_resort_enriched.csv'
df_clean.to_csv(output_path, index=False)
print(f"✅ Enriched dataset exported to: {output_path}")

# Export recommendations
rec_df = pd.DataFrame(recommendations)
rec_path = '/mnt/user-data/outputs/recommendations.csv'
rec_df.to_csv(rec_path, index=False)
print(f"✅ Recommendations exported to: {rec_path}")

# Export feature importance
fi_path = '/mnt/user-data/outputs/feature_importance.csv'
feature_importance.to_csv(fi_path, index=False)
print(f"✅ Feature importance exported to: {fi_path}")

print("\n📊 All results have been exported successfully!")